# <center> CAO Points Analysis </center>
### Conor McCaffrey

***

![cao picture](Images/cao.png)

<i><sub> Picture adapted from Reference 1 </sub></i>
<br>


Please click the following link to go to the CAO website:
[CAO Website](https://www.cao.ie)

This notebook has two dual purposes: 
* To compare and contrast the CAO points in 2019,2020 and 2021 using the functionality of <i>Pandas</i>.
* To display knowledge gained throughout the duration of the course, in this case displayed through the use of plots and other visualisations of the above data.

***



### Table of Contents
* Loading [2021 Points](#2021_Points) Dataset
* Loading [2020 Points](#2020_Points) Dataset 
* Loading [2019 Points](#2019_Points) Dataset 
* Analysis of Level 8 2021/2019 and 2020 Points
*
*



         


### Let's import our packages

In [1]:
# Convenient HTTP requests.
import requests as rq

# Regular expressions.
import re

# Dates and times.
import datetime as dt

# Data frames. (dataframes is another way to say spreadsheets)
import pandas as pd

# For downloading.
import urllib.request as urlrq

# For plotting
import seaborn as sns

# For plotting
import matplotlib.pyplot as plt



<a id='2021_Points'></a>
<br>

## 2021 Points

#### Please click below for 2021 courses
http://www.cao.ie/index.php?page=points&p=2021




Give overview here of response function (concise)

##### Steps to re-produce loading of Data in Pandas Dataframe

1. Use Requests library [3] to make HTTP request to Python.
2. Using DateTime package to get current date and time [4], create a filepath for data.
3. Rectify encoding error on dataset by setting to 'cp1252'.
4. Re-save Word Document for editing.
5. Save this rectifed HTML dataset.
6. Using regular expressions [5], select the information we require from HTML file.
7. Loop through this file, using regular expression and save as a .csv file.
8. Read this .csv file into a Pandas dataframe
9. Call a method on the dataframe to ensure it is loaded correctly



##### Let's gather the Level 8 courses

In [2]:
# Fetch the CAO Level 8 points URL.  (based on Lecture Notes)
resp8 = rq.get('http://www2.cao.ie/points/l8.php')
# Have a quick look to ensure all is okay
resp8       # Response 200 means all is okay. 404 means not found

<Response [200]>

##### Let's gather the Level 7/6 courses

In [3]:
# Fetch the CAO Level 7/6 points URL (adapted from Lecture Notes)
resp7 = rq.get('http://www2.cao.ie/points/l76.php')
# Have a quick look to ensure all is okay
resp7      # Response 200 means all is okay. 404 means not found

<Response [200]>

<br>

## Save original dataset

In [4]:
# Get current date and time using Datetime Package [4]
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [5]:
# Create a filepath for original Level 8 data (Based on reference 6)
path_HTML8 = 'data/cao2021_Lvl8_' +  nowstr + '.html'
# Create a filepath for original Level 7/6 data
path_HTML7 = 'data/cao2021_Lvl76_' +  nowstr + '.html'

<br>

## Charset error on CAO server

***

Technically the server says we should decode as iso-8859:

```
Content-Type: text/html; charset=iso-8859-1

```

However, one line uses \x96 which isnt defined in iso-8859-1. <br>
Therefore, we use the similar decoding standard cp1252 which is similar but also includes \x96.

In [6]:
## Level 8
# Accounting for the error on the server
original_encoding = resp8.encoding
# Change it to cp1252
resp8.encoding = 'cp1252'

In [7]:
# Save the original HTML file
with open(path_HTML8, 'w') as f:
    f.write(resp8.text)

In [8]:
## Level 7/6
# Accounting for the error on the server for Level 7/6 data
original_encoding = resp7.encoding
# Change it to cp1252
resp7.encoding = 'cp1252'

In [9]:
# Save the original HTML file
with open(path_HTML7, 'w') as f:
    f.write(resp7.text)

# Use regular expressions to select lines we want

***

In [10]:
# Compile the regular expression for matching lines. [5]
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)') 


### Loop through the lines of the response and save result as a dataframe

***

In [11]:
#The filepath for the csv file. Level 8
path2021_8 = 'data/cao2021_Lvl8_' +  nowstr + '.csv'



In [12]:
# keep track of how many courses we process  COMMENT THIS
no_lines = 0

# Open the csv file for writing.
try:
    with open (path2021_8, 'w') as f:
        # Write a header row.
        f.write(','.join(['Code', 'Title', 'PointsR1', 'PointsR2']) + '\n')
        # Loop through the lines of the response.
        for line in resp8.iter_lines(): # took from google stack overflow.
        # Decode line, using the wrong encoding!
            dline = line.decode('cp1252')
        # match only the lines we want - the ones representing courses.
        # getting set up for regular expression. 'pattern is the regular expression itself'
            if re_course.fullmatch(dline):   
            # Add one to the lines counter
                no_lines = no_lines + 1
            # The course code
                course_code = dline[:5]
            # The course title
                course_title = dline[7:57].strip() # Remove whitespace
            # Round 1 points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
            # Join the fields using a comma.
            # make final two elements into arrays with 3 things.
                linesplit = [course_code, course_title, course_points[0], course_points[1]] 
            # rejoin the substrings with commas in between.
                f.write(','.join(linesplit) + '\n')
except BaseException as err:
    print('This has caused an error')
else:
    print('Process has been successful.',f'Total number of lines is {no_lines}.')          




Process has been successful. Total number of lines is 949.


<br>

<b> ** N.B ** </b> It was verified as of 08/11/2021 there were 949 courses exactly in the CAO 2021 Points List.

***

In [13]:
# Saving our result in a dataframe [7]
df2021_8 = pd.read_csv(path2021_8, encoding='cp1252') 

# consider making a table listing University and the letters for it

In [14]:
# Calling variable to test
df2021_8

,Code,Title,PointsR1,PointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


***
## <u> Now let's complete the same for Level 7/6 Data </u>
 <br>
 
 ### Loop through the lines of the response and save result as a dataframe


In [15]:
#The filepath for the csv file.

path2021_7 = 'data/cao2021_Lvl76_' +  nowstr + '.csv'
# keep track of how many courses we process
no_lines = 0

# Open the csv file for writing.
try:
    with open (path2021_7, 'w') as f:
        # Write a header row.
        f.write(','.join(['Code', 'Title', 'PointsR1', 'PointsR2']) + '\n')
        # Loop through the lines of the response.
        for line in resp7.iter_lines(): # took from google stack overflow.
            # Decode line, using the wrong encoding!
            dline = line.decode('cp1252')
            # match only the lines we want - the ones representing courses.
            if re_course.fullmatch(dline):   
                # Add one to the lines counter
                no_lines = no_lines + 1
        
                # The course code
                course_code = dline[:5]
                # The course title
                course_title = dline[7:57].strip()
                # Round 1 points
                course_points = re.split(' +', dline[60:])
                if len(course_points) != 2:
                    course_points = course_points[:2]
                # Join the fields using a comma.
                # make last two things into arrays with 3 things.
                linesplit = [course_code, course_title, course_points[0], course_points[1]] 
                # rejoin the substrings with commas in between.
                f.write(','.join(linesplit) + '\n')
except BaseException as err:
    print('This has caused an error')
else:
    print('Process has been successful.',f'Total number of lines is {no_lines}.')    


Process has been successful. Total number of lines is 416.


<br>

<b> ** N.B ** </b> It was verified as of 23/11/2021 there were 416 courses exactly in the CAO 2021 Points List for Level 7/6.

***

In [16]:
# Saving our result in a dataframe [7]

df2021_7 = pd.read_csv(path2021_7, encoding='cp1252') 

In [17]:
# lets ensure it calls out correctly
df2021_7

,Code,Title,PointsR1,PointsR2
0,AL605,Music and Instrument Technology,211,NaN
1,AL630,Pharmacy Technician,308,NaN
2,AL631,Dental Nursing,311,NaN
3,AL632,Applied Science,297,NaN
4,AL650,Business,AQA,AQA
...,...,...,...,...
411,WD188,Applied Health Care,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v
413,WD206,Electronic Engineering,180,NaN
414,WD207,Mechanical Engineering,172,NaN


<br>
<a id='2020_Points'></a>

## 2020 Points

#### Please click below for 2020 courses
http://www.cao.ie/index.php?page=points&p=2020





***

In [18]:

# Saving URL for 2020 points to the variable url2020
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

##### Steps to re-produce loading of Data in Pandas Dataframe

1. Using DateTime package, create filepath for data in excel format.
2. Save this file to disk using 'path' variable and url requests (https://docs.python.org/3/library/urllib.request.html).
3. Read data into Pandas Dataframe [7], using skiprows to remove unecessary data.
4. Spot check the data to ensure it has all transferred correctly.
5. Creat filepath for dataframe in excel .csv format.
6. Save Pandas dataframe to disk.


<br>

### Save Original Dataset

***

#### Let's gather the Data

In [19]:
# Create a filepath for original data. 
path2020 = 'data/cao2020_' +  nowstr + '.xlsx'

In [20]:
# Save original file to disk. [8]
urlrq.urlretrieve(url2020, path2020) 

('data/cao2020_20211125_221413.xlsx',
 <http.client.HTTPMessage at 0x21f2e677af0>)

<br>

#### Load Spreadsheet using Pandas

***

In [21]:
# Download and parse the excel spreadsheet
# Read excel function turns it into a dataframe in memory so we are now dealing with pandas df from here
df2020 = pd.read_excel(url2020, skiprows = 10) # pandas dataframe

In [22]:
# Calling the variable to test
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

<b> ** N.B ** </b> It was verified as of 23/11/2021 there were 1464 courses exactly in the CAO 2020 Points List.

***

In [23]:
# Testing our dataframe. iloc retrieves row location [9]
df2020.iloc[753]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Road Transport Technology and Management
COURSE CODE2                                                           LC286
R1 POINTS                                                                264
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      264
EOS Random *                                                             NaN
EOS Mid-point                                                            360
LEVEL                                                                      7
HEI                                         Limerick Institute of Technology
Test/Interview #                                                         NaN

In [24]:
# Testing final row
df2020.iloc[1463]  # spot checks

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [25]:
# Create a filepath for the Pandas data
path = 'data/cao2020_' +  nowstr + '.csv'

In [26]:
# Save Pandas dataframe to disk. Look up to_csv to get rid of row index
df2020.to_csv(path)

***
<br>
<a id='2019_Points'></a>


## 2019 Points

<i> Please [Click Here](http://www.cao.ie/index.php?page=points&p=2019 "click Here") for 2019 points </i>



##### Steps to re-produce this work

1. Download original PDF file.
2. Open original PDF file in Microsoft Word.
3. Save Microsoft Word's converted PDF in docx format.
4. Re-save Word Document for editing.
5. Delete Headers and Footers.
6. Delete preamble on Page 1.
7. Select all and copy.
8. Paste into Notepad ++.
9. Remove HEI name headings and paste onto each course line.
10. Delete blank lines.
11. Replace double tab characters with a single tab character (6 occurences)
12. Deleted tab characters at end of lines
13. Change backticks to apostrophes

## Lets prepare our Level 8 Data

In [27]:
# Constructing our dataframe
df2019 = pd.read_csv('data/cao2019_20211108_223304.csv', sep = '\t') # separate on tabs
# Call our new variable
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


<br>

<b> ** N.B ** </b> It was verified as of 23/11/2021 there were 930 courses exactly in the CAO 2019 Points List for Level 8.

***

In [28]:
# Spot checks
df2019.iloc[900:926]

,Course Code,Course,EOS,Mid
900,WD048,Business (options),278,357.0
901,WD084,Accounting,307,403.0
902,WD091,Hospitality Management,210,281.0
903,WD116,General Nursing,413,430.0
904,WD117,Psychiatric Nursing,361,382.0
905,WD120,Intellectual Disability Nursing,338,359.0
906,WD134,International Business,289,350.0
907,WD137,Design (Visual Communications),253,373.0
908,WD140,Laws,290,352.0
909,WD144,Architecture,283,385.0


## Lets prepare our Level 7/6 Data

In [29]:
# Reading in our csv file
df2019_l7 = pd.read_csv('data/cao2019_20211101_221930_edited.CSV', sep = '\t')

In [30]:
# Spot checks
df2019.iloc[900:926]

,Course Code,Course,EOS,Mid
900,WD048,Business (options),278,357.0
901,WD084,Accounting,307,403.0
902,WD091,Hospitality Management,210,281.0
903,WD116,General Nursing,413,430.0
904,WD117,Psychiatric Nursing,361,382.0
905,WD120,Intellectual Disability Nursing,338,359.0
906,WD134,International Business,289,350.0
907,WD137,Design (Visual Communications),253,373.0
908,WD140,Laws,290,352.0
909,WD144,Architecture,283,385.0


In [31]:
# Sorting the values
df2019_l7.sort_values('Course Code')

,Course Code,Course,EOS,Mid
0,AL600,Software Design,205,306.0
1,AL601,Computer Engineering,196,272.0
2,AL602,Mechanical Engineering,258,424.0
3,AL604,Civil Engineering,252,360.0
4,AL630,Pharmacy Technician,306,366.0
...,...,...,...,...
456,WD188,Applied Health Care,206,339.0
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0
458,WD206,Electronic Engineering,191,322.0
459,WD207,Mechanical Engineering,179,330.0


<br>

<b> ** N.B ** </b> It was verified as of 23/11/2021 there were 461 courses exactly in the CAO 2019 Points List for Level 7/6.

***

## <u> Analysis of CAO 2021 Level 8,7,6 Points </u>


<br>

## Lets join our imported Datasets for Level 8 data and 2020 data

In [32]:
# Lets concatenate all Round 1 points from 2021-2019
courses2021_8 = df2021_8[['Code', 'Title']] # Access columns we are interested in

# Lets repeat the above work for 2020
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['Code','Title']
courses2020.sort_values('Code')  # Display values, sorting on 'Code'


# Lets gather our Round 1 courses for 2019
courses2019_8 = df2019[['Course Code','Course']]
courses2019_8.columns = ['Code','Title']
courses2019_8.sort_values('Code')  # Display values, sorting on 'Code'


# Lets concatenate our results [10]
allcourses_L8 = pd.concat([courses2021_8, courses2020, courses2019_8], ignore_index=True) # Based on Lecture Notes


# Some manipulation of result, display result sorting on 'Code'
allcourses_L8.sort_values('Code')


# Finds all extra copies of duplicated rows
allcourses_L8[allcourses_L8.duplicated()]

# Returns a copy of the data frame with duplicates removed.
allcourses_L8.drop_duplicates()


# Finds all extra copies of duplicated rows
allcourses_L8[allcourses_L8.duplicated(subset =['Code'])]



# Returns a copy of the data frame with duplicates removed = based only on code [11]
allcourses_L8.drop_duplicates(subset = ['Code'], inplace = True, ignore_index = True)


## Pandas Join [13]


***

In [33]:
# Lets access 2021 Level 8 Data
df2021_8

,Code,Title,PointsR1,PointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [34]:
# Set the index to the code column
df2021_8.set_index('Code', inplace = True)
df2021_8.columns = ['Title','Points_R1_2021', 'Points_R2_2021']
df2021_8

,Title,Points_R1_2021,Points_R2_2021
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [35]:
# Set the index to the code column and for Round 1 Data
allcourses_L8.set_index('Code', inplace = True)
allcourses_L8 = allcourses_L8.join(df2021_8[[ 'Points_R1_2021']])
allcourses_L8

,Title,Points_R1_2021
Code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
SG441,Environmental Science,NaN
SG446,Applied Archaeology,NaN
TL803,Music Technology,NaN


In [36]:
# Lets access 2020 Data
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Set the index to the code column and for Round 1 Data
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns=['Code', 'Points_R1_2020']
df2020_r1.set_index('Code', inplace = True)
df2020_r1

,Points_R1_2020
Code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [38]:
# Join 2020 points to allcourses_L8
allcourses_L8 = allcourses_L8.join(df2020_r1)
allcourses_L8

,Title,Points_R1_2021,Points_R1_2020
Code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
SG441,Environmental Science,NaN,NaN
SG446,Applied Archaeology,NaN,NaN
TL803,Music Technology,NaN,NaN


#### 2019 Join

In [39]:
# Set the index to the code column and for Round 1 Data
df2019_r1 = df2019[['Course Code', 'Mid']]
df2019_r1.columns = ['Code', 'Points_R1_2019']
df2019_r1.set_index('Code', inplace = True)
df2019_r1

,Points_R1_2019
Code,
AL801,328.0
AL802,306.0
AL803,337.0
AL805,442.0
AL810,349.0
...,...
WD200,296.0
WD210,329.0
WD211,322.0


In [40]:
# Join 2019 points to allcourses_l8 and display the dataframe
allcourses_L8 = allcourses_L8.join(df2019_r1)
allcourses_L8

,Title,Points_R1_2021,Points_R1_2020,Points_R1_2019
Code,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,328.0
AL802,Software Design in Artificial Intelligence for...,313,332,306.0
AL803,Software Design for Mobile Apps and Connected ...,350,337,337.0
AL805,Computer Engineering for Network Infrastructure,321,333,442.0
AL810,Quantity Surveying,328,319,349.0
...,...,...,...,...
SG441,Environmental Science,NaN,NaN,358.0
SG446,Applied Archaeology,NaN,NaN,290.0
TL803,Music Technology,NaN,NaN,288.0


We now have a dataframe that contains all R1 points for CAO courses from 2021-2019.

## Lets repeat this work for Level 7/6 Data for 2021 & 2019

In [41]:
# Lets concatenate all Round 1 points from 2021 and 2019
courses2021_7 = df2021_7[['Code', 'Title']] # Access columns we are interested in


# Lets gather our Round 1 courses for 2019
courses2019_7 = df2019_l7[['Course Code','Course']]
courses2019_7.columns = ['Code','Title']
courses2019_7.sort_values('Code')  # Display values, sorting on 'Code'


# Lets concatenate our results [10]
allcourses_L7 = pd.concat([courses2021_7, courses2019_7], ignore_index=True) # Based on Lecture Notes


# Some manipulation of result, display result sorting on 'Code'
allcourses_L7.sort_values('Code')


# Finds all extra copies of duplicated rows
allcourses_L7[allcourses_L7.duplicated()]

# Returns a copy of the data frame with duplicates removed.
allcourses_L7.drop_duplicates()


# Finds all extra copies of duplicated rows
allcourses_L7[allcourses_L7.duplicated(subset =['Code'])]



# Returns a copy of the data frame with duplicates removed = based only on code [11]
allcourses_L7.drop_duplicates(subset = ['Code'], inplace = True, ignore_index = True)

In [42]:
# Lets access 2021 Data
df2021_7

,Code,Title,PointsR1,PointsR2
0,AL605,Music and Instrument Technology,211,NaN
1,AL630,Pharmacy Technician,308,NaN
2,AL631,Dental Nursing,311,NaN
3,AL632,Applied Science,297,NaN
4,AL650,Business,AQA,AQA
...,...,...,...,...
411,WD188,Applied Health Care,220,NaN
412,WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v
413,WD206,Electronic Engineering,180,NaN
414,WD207,Mechanical Engineering,172,NaN


In [43]:
# Set the index to the code column and for Round 1 Data
df2021_7.set_index('Code', inplace = True)
df2021_7.columns = ['Title','Points_R1_2021', 'Points_R2_2021']
df2021_7

,Title,Points_R1_2021,Points_R2_2021
Code,,,
AL605,Music and Instrument Technology,211,NaN
AL630,Pharmacy Technician,308,NaN
AL631,Dental Nursing,311,NaN
AL632,Applied Science,297,NaN
AL650,Business,AQA,AQA
...,...,...,...
WD188,Applied Health Care,220,NaN
WD205,Molecular Biology with Biopharmaceutical Science,AQA,262v
WD206,Electronic Engineering,180,NaN


In [44]:
# Set the index to the code column and for Round 1 Data
allcourses_L7.set_index('Code', inplace = True)
allcourses_L7 = allcourses_L7.join(df2021_7[[ 'Points_R1_2021']])
allcourses_L7

,Title,Points_R1_2021
Code,,
AL605,Music and Instrument Technology,211
AL630,Pharmacy Technician,308
AL631,Dental Nursing,311
AL632,Applied Science,297
AL650,Business,AQA
...,...,...
TL620,Culinary Arts,NaN
TL712,Computing with Digital Media,NaN
TL719,Culinary Arts,NaN


In [45]:
# Lets access 2019 Data
df2019_l7

,Course Code,Course,EOS,Mid
0,AL600,Software Design,205,306.0
1,AL601,Computer Engineering,196,272.0
2,AL602,Mechanical Engineering,258,424.0
3,AL604,Civil Engineering,252,360.0
4,AL630,Pharmacy Technician,306,366.0
...,...,...,...,...
456,WD188,Applied Health Care,206,339.0
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0
458,WD206,Electronic Engineering,191,322.0
459,WD207,Mechanical Engineering,179,330.0


In [46]:
# Set the index to the code column and for Round 1 Data
df2019_l7 = df2019_l7[['Course Code', 'Mid']]
df2019_l7.columns = ['Code', 'Points_Mid_2019']
df2019_l7.set_index('Code', inplace = True)
df2019_l7

,Points_Mid_2019
Code,
AL600,306.0
AL601,272.0
AL602,424.0
AL604,360.0
AL630,366.0
...,...
WD188,339.0
WD205,441.0
WD206,322.0


In [47]:
# Set the index to the code column and for Round 1 Data
allcourses_L7 = allcourses_L7.join(df2019_l7[[ 'Points_Mid_2019']])
allcourses_L7

,Title,Points_R1_2021,Points_Mid_2019
Code,,,
AL605,Music and Instrument Technology,211,NaN
AL630,Pharmacy Technician,308,366.0
AL631,Dental Nursing,311,379.0
AL632,Applied Science,297,372.0
AL650,Business,AQA,317.0
...,...,...,...
TL620,Culinary Arts,NaN,234.0
TL712,Computing with Digital Media,NaN,352.0
TL719,Culinary Arts,NaN,368.0


We now have a dataframe that contains all R1 points for CAO courses for 2021 and 2019 Level 7/6. It was not feasible to include 2020 in this category as all raw data is contained on the same spreadsheet, which is a shame. 

## Descriptive Statisitcs on our newly-minted Dataframes

In [48]:
allcourses_L7

,Title,Points_R1_2021,Points_Mid_2019
Code,,,
AL605,Music and Instrument Technology,211,NaN
AL630,Pharmacy Technician,308,366.0
AL631,Dental Nursing,311,379.0
AL632,Applied Science,297,372.0
AL650,Business,AQA,317.0
...,...,...,...
TL620,Culinary Arts,NaN,234.0
TL712,Computing with Digital Media,NaN,352.0
TL719,Culinary Arts,NaN,368.0


In [49]:
allcourses_L8

,Title,Points_R1_2021,Points_R1_2020,Points_R1_2019
Code,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,328.0
AL802,Software Design in Artificial Intelligence for...,313,332,306.0
AL803,Software Design for Mobile Apps and Connected ...,350,337,337.0
AL805,Computer Engineering for Network Infrastructure,321,333,442.0
AL810,Quantity Surveying,328,319,349.0
...,...,...,...,...
SG441,Environmental Science,NaN,NaN,358.0
SG446,Applied Archaeology,NaN,NaN,290.0
TL803,Music Technology,NaN,NaN,288.0


In [50]:
# Size of our dataframe
shape = allcourses_L8.shape
print("Data frame shape", shape, "rows,columns")

# Print the first five lines.
allcourses_L8.head()

Data frame shape (1651, 4) rows,columns


,Title,Points_R1_2021,Points_R1_2020,Points_R1_2019
Code,,,,
AL801,Software Design for Virtual Reality and Gaming,300,303,328.0
AL802,Software Design in Artificial Intelligence for...,313,332,306.0
AL803,Software Design for Mobile Apps and Connected ...,350,337,337.0
AL805,Computer Engineering for Network Infrastructure,321,333,442.0
AL810,Quantity Surveying,328,319,349.0


In [51]:
# Size of our dataframe
shape = allcourses_L7.shape
print("Data frame shape", shape, "rows,columns")

# Print the first five lines.
allcourses_L7.head()

Data frame shape (554, 3) rows,columns


,Title,Points_R1_2021,Points_Mid_2019
Code,,,
AL605,Music and Instrument Technology,211,NaN
AL630,Pharmacy Technician,308,366.0
AL631,Dental Nursing,311,379.0
AL632,Applied Science,297,372.0
AL650,Business,AQA,317.0


Our dataframes have been constructed so that they are quite visually accessible. We have our Course Code (Code), our Course Title (Title), and our Round 1 points for each applicable year. The Level 8 data contains 1651 rows and our Level 7/6 data contains 554 rows. 

In [52]:
print('This is the information regarding Level 8 courses')
info1 = allcourses_L8.info()
print()
print('This is the information regarding Level 7 courses')
info2 = allcourses_L7.info()


This is the information regarding Level 8 courses
<class 'pandas.core.frame.DataFrame'>
Index: 1651 entries, AL801 to WD148
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           1651 non-null   object 
 1   Points_R1_2021  928 non-null    object 
 2   Points_R1_2020  1437 non-null   object 
 3   Points_R1_2019  915 non-null    float64
dtypes: float64(1), object(3)
memory usage: 129.0+ KB

This is the information regarding Level 7 courses
<class 'pandas.core.frame.DataFrame'>
Index: 554 entries, AL605 to WD139
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            554 non-null    object 
 1   Points_R1_2021   408 non-null    object 
 2   Points_Mid_2019  453 non-null    float64
dtypes: float64(1), object(2)
memory usage: 33.5+ KB


In [73]:
# It appears we have NaN values in our datasets, let's confirm this
check_NaN_in_df = allcourses_L8.isnull().values.any()
print (check_NaN_in_df) # Boolean to determine presence of NaN values
check_NaN_in_df = allcourses_L8.isnull()
print (check_NaN_in_df) # Verifying this with a snapshot of data
# Count of the NaN Values
print('We have' ,allcourses_L8.isnull().sum().sum(), 'NaN values in our dataset' )

True
       Title  Points_R1_2021  Points_R1_2020  Points_R1_2019  value_is_NaN
Code                                                                      
AL801  False           False           False           False         False
AL802  False           False           False           False         False
AL803  False           False           False           False         False
AL805  False           False           False           False         False
AL810  False           False           False           False         False
...      ...             ...             ...             ...           ...
SG441  False            True            True           False         False
SG446  False            True            True           False         False
TL803  False            True            True           False         False
TL812  False            True            True           False         False
WD148  False            True            True           False         False

[1651 rows x 5 colu

In [74]:
# It appears we have NaN values in our datasets, let's confirm this
check_NaN_in_df = allcourses_L7.isnull().values.any()
print (check_NaN_in_df)
check_NaN_in_df = allcourses_L7.isnull()
print (check_NaN_in_df)
# Count of the NaN Values
print('We have' ,allcourses_L7.isnull().sum().sum(), 'NaN values in our dataset' )

True
       Title  Points_R1_2021  Points_Mid_2019
Code                                         
AL605  False           False             True
AL630  False           False            False
AL631  False           False            False
AL632  False           False            False
AL650  False           False            False
...      ...             ...              ...
TL620  False            True            False
TL712  False            True            False
TL719  False            True            False
TL746  False            True             True
WD139  False            True            False

[554 rows x 3 columns]
We have 247 NaN values in our dataset


In [86]:
# Investigating the two NaN values I've spotted for a course to confirm
allcourses_L7.iloc[552]

Title              Engineering in Mechatronics Engineering
Points_R1_2021                                         NaN
Points_Mid_2019                                        NaN
Name: TL746, dtype: object

<b>N.B</b> I have confirmed that result is indeed correct by verifying from raw data

In [54]:
# Ideas

Most points , least points
Biggest change year on year (between all)
Which college offers most course
How many courses offer placements in title
how many courses had Round 2
which courses accepted anyone
Level 7 no points for some courses
how many courses in points range x-x, 
What colleges had most points for engineering in title, science in title
did any courses cease operating YOY